In [1]:
import pandas as pd

In [2]:
%pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-snjykkez
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ | done
    Preparing wheel metadata ... - \ | / - done
     |████████████████████████████████| 764 kB 15.7 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 6.5 MB/s  eta 0:00:01
     |████████████████████████████████| 895 kB 69.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.5 MB 99.3 MB/s eta 0:00:01
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3958151 sha256=5af97936d52a24b0a9c79cf3b9901516531ddd8bb527254e03a844e86703da64
  Stored in directory: /tmp/pip-ephem-wheel-cache-qft01a_a/wheels/05/0a/97/64ae47c27ba95fae2cb5838e7b4b7247a34d4a8ba5f7092de2
Successfully built transformers
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import
from transformers import GPT2Tokenizer, GPTNeoForCausalLM 
import re

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# model name avec colab pro
#model_name = "EleutherAI/gpt-neo-2.7B"

#model name avec colab standard
model_name = "EleutherAI/gpt-neo-1.3B"


# load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


Downloading: 100%|██████████| 779k/779k [00:00<00:00, 1.65MB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 1.40MB/s]
Downloading: 100%|██████████| 90.0/90.0 [00:00<00:00, 86.1kB/s]
Downloading: 100%|██████████| 200/200 [00:00<00:00, 148kB/s]
Downloading: 100%|██████████| 1.32k/1.32k [00:00<00:00, 1.28MB/s]


In [5]:
model = GPTNeoForCausalLM.from_pretrained(model_name)

Downloading: 100%|██████████| 4.95G/4.95G [00:57<00:00, 92.1MB/s]


In [6]:
def generate(text, min_length=10, max_length=100, remove_input=True,
                   end_sequence=None, top_k=0, top_p=0.7, num_return_sequences=1,
                   temperature=1, repetition_penalty=1, length_penalty=1):
  tokens = tokenizer.tokenize(text)
  print("Input number of tokens: ", len(tokens))
  output_max_length = len(tokens)+ max_length
  # generate tokens
  input_ids = tokenizer(text, return_tensors="pt").input_ids
  # perform prediction 
  gen_tokens = model.generate(input_ids, do_sample=True, max_length=output_max_length, top_k=top_k, 
                              top_p=top_p, temperature=temperature, repetition_penalty=repetition_penalty, length_penalty=length_penalty, num_return_sequences=num_return_sequences)
  # decode the predicted tokens into texts
  predicted_text = tokenizer.batch_decode(gen_tokens)[0]
  if remove_input:
    predicted_text = predicted_text[len(text):]
  # A simple technique to stop at stop_sequence without modifying the underlying model
  if end_sequence is not None and end_sequence in text:
        predicted_text = predicted_text.split(end_sequence)[0]
  return predicted_text


In [13]:
df = pd.read_csv("df_valid_to_train_fs.csv")

In [8]:
initial_prompt = """Find syllogism conclusion

Premise 1: All gamblers are bakers.
Premise 2: All bakers are sculptors.
Conclusion: All gamblers are sculptors.
###
Premise 1: All rabbits are cleaners.
Premise 2: Some analysts are rabbits.
Conclusion: Some analysts are cleaner.
###
Premise 1: No reptiles are joggers.
Premise 2: All tutors are reptiles.
Conclusion: No tutors are joggers.
###
Premise 1: Some cats are not mammals.
Premise 2: All pets are cats.
Conclusion: Some pets are not mammals.
### """

In [9]:
def laprediction(sentence, initial_prompt):
    sentence = sentence.replace(" and", ".\nPremise 2:")
    sentence = "Premise 1: " + sentence + ".\nConclusion:"
    
    final_prompt = initial_prompt + "\n" + sentence
    #return final_prompt
    prediction_syllo = generate(final_prompt, end_sequence='###', max_length=200)
    return prediction_syllo[:-2]


In [14]:
df['few_shot_pred'] = df.sentenced.apply(lambda x : laprediction(x,initial_prompt))
df.to_csv("few_shot_trained_full.csv" ,index=False)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Input number of tokens:  163
Input number of tokens:  163
Input number of tokens:  168
Input number of tokens:  164
Input number of tokens:  165
Input number of tokens:  168
Input number of tokens:  163
Input number of tokens:  168
Input number of tokens:  164
Input number of tokens:  165
Input number of tokens:  168
Input number of tokens:  163
Input number of tokens:  168
Input number of tokens:  164
Input number of tokens:  165
Input number of tokens:  168
Input number of tokens:  166
Input number of tokens:  163
Input number of tokens:  165
Input number of tokens:  164
Input number of tokens:  163
Input number of tokens:  163
Input number of tokens:  164
Input number of tokens:  167
Input number of tokens:  165
Input number of tokens:  164
Input number of tokens:  164
Input number of tokens:  167
Input number of tokens:  163
Input number of tokens:  167
Input number of tokens:  167
Input number of tokens:  164
Input number of tokens:  167
Input number of tokens:  165
Input number o